In [ ]:
import json
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
from config import db_password
#import psycopg2

In [ ]:
def etl_process (wiki_data, kaggle_data, rating_data):
    
    #eliminate all data that is not a movie
    wiki_movies= [movie for movie in wiki_movies
             if ('Director' in movie or 'Directed by' in movie
                and 'imdb_link' in movie
                and 'No. of episodes' not in movie)]
    
    #create alternative title column
    
    def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    #make an alternative dict to hold key: value pair
    alt_titles= {}
    #loop through all the alternative titles
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        #check if the current key exits in the movie object (the column)
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    #add alternative titles dict to the movie object 
    if len(alt_titles)>0:
        movie['alt_titles']= alt_titles
    
    
    # merge column names
        def change_column_name(old_name, new_name):
            if old_name in movie:
                movie[new_name] = movie.pop(old_name)
        change_column_name('Adaptation by', 'Writer(s)')
        change_column_name('Created by', 'Creator(s)')
        change_column_name('Country of origin', 'Country')
        change_column_name('Directed by', 'Director')
        change_column_name('Distributed by', 'Distributor')
        change_column_name('Edited by', 'Editor(s)')
        change_column_name('Length', 'Running time')
        change_column_name('Original release', 'Release date')
        change_column_name('Music by', 'Composer(s)')
        change_column_name('Narrated by', 'Narrator(s)')
        change_column_name('Produced by', 'Producer(s)')
        change_column_name('Producer', 'Producer(s)')
        change_column_name('Productioncompanies ', 'Production company(s)')
        change_column_name('Productioncompany ', 'Production company(s)')
        change_column_name('Released', 'Release Date')
        change_column_name('Release Date', 'Release date')
        change_column_name('Screen story by', 'Writer(s)')
        change_column_name('Screenplay by', 'Writer(s)')
        change_column_name('Story by', 'Writer(s)')
        change_column_name('Theme music composer', 'Composer(s)')
        change_column_name('Written by', 'Writer(s)')
        change_column_name('alt_titles', 'Alternative Titles')
    
    return movie
    
    #apply clean movies to wiki_movies
    clean_movies = [clean_movie(movie) for movie in wiki_movies]
    #get a dataframe
    wiki_movies_df= pd.DataFrame